In [1]:
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point, Polygon
import tkinter as tk
from tkinter import filedialog
print('All imported')

All imported


функция вызова всплывающего окна

In [2]:
def get_working_directory():
    root = tk.Tk()
    root.title("Выбор рабочей папки")
    root.geometry("500x150")
    root.resizable(False, False)
    
    folder_path = tk.StringVar()
    
    def browse_folder():
        folder = filedialog.askdirectory()
        if folder:
            folder_path.set(folder)
            entry.delete(0, tk.END)
            entry.insert(0, folder)
    
    def submit():
        root.destroy()
    
    tk.Label(root, text="Введите рабочую папку:", font=("Arial", 12)).pack(pady=(10, 5))
    
    input_frame = tk.Frame(root)
    input_frame.pack(fill=tk.X, padx=20, pady=5)
    
    entry = tk.Entry(input_frame, textvariable=folder_path, font=("Arial", 12))
    entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
    
    browse_btn = tk.Button(input_frame, text="Обзор", command=browse_folder, width=10)
    browse_btn.pack(side=tk.RIGHT)
    
    submit_btn = tk.Button(root, text="OK", command=submit, width=15, font=("Arial", 10))
    submit_btn.pack(pady=10)
    
    root.mainloop()
    return folder_path.get()

запуск всплывающего диалогового окна

In [3]:
if __name__ == "__main__":
    papka = get_working_directory()

Функция calculate_radius_of_zrv(): min_delta_angle = минимальный угол зоны радиовидимости (градусы); alt = высота орбиты КА над ППИ (км); radius_of_earth -- радиус Земли (базовое значение = 6371.032 км)

In [4]:
def calculate_radius_of_zrv(min_delta_angle=5, alt=518833, radius_of_earth=6371032):
    angle_of_zone_of_radiovision = math.acos((math.cos(min_delta_angle * (math.pi / 180)) * radius_of_earth) / (alt + radius_of_earth)) - (min_delta_angle * (math.pi / 180))
    return angle_of_zone_of_radiovision * radius_of_earth

In [5]:
def genaral_zrv(ppi_id, file_input, file_output):
    PPI = pd.read_csv(file_input)
    ppi_gdf = gpd.GeoDataFrame(PPI, geometry=gpd.points_from_xy(PPI.Longitude, PPI.Latitude))
    for id, row in ppi_gdf.iterrows():
        if id not in ppi_id:
            ppi_gdf.drop(axis=0, index=id, inplace=True)
    ppi_gdf.crs="EPSG:4326"
    ppi_converted = ppi_gdf.to_crs(epsg=3043)
    ppi_converted["geometry"] = ppi_converted.buffer(calculate_radius_of_zrv())
    ppi_result = ppi_converted.to_crs(epsg=4326)
    ppi_result.to_file(file_output + '//GENERAL_PPI.shp')

Функция make_zrv(): file_input -- абсолютный путь к файлу с ППИ; file_output = абсолютный путь к выходному шейп файлу; ppi_id = массив идентификаторов ППИ в исходном файле, которые нужны для работы [0-11]

In [6]:
def make_zrv(ppi_id=[], file_input=papka + '//ppi.csv', file_output=papka):
    PPI = pd.read_csv(file_input)
    ppi_gdf = gpd.GeoDataFrame(PPI, geometry=gpd.points_from_xy(PPI.Longitude, PPI.Latitude))
    ppi_gdf.crs = "EPSG:4326"
    ppi_gdf.to_file(file_output + '//PPI_POINTS.shp')
    ppi_converted = ppi_gdf.to_crs(epsg=3043)
    ppi_converted["geometry"] = ppi_converted.buffer(calculate_radius_of_zrv())
    ppi_result = ppi_converted.to_crs(epsg=4326)
    genaral_zrv(ppi_id=ppi_id, file_input=file_input, file_output=file_output)
    step = 0
    for i in range(len(ppi_result)):
        ppi_table = ppi_result
        for id, row in ppi_table.iterrows():
            if id != i:
                ppi_table = ppi_table.drop(axis=0, index=id, inplace=False)
        if i in ppi_id:
            ppi_table.to_file(file_output + '//test_ppi' + str(step) + '.shp')
        step += 1

In [7]:
make_zrv(ppi_id=list(range(12)))
# make_zrv([0, 1, 2, 3, 10, 11])

In [8]:
def intersect_subsatellite_points_with_zrv(input_zrv=papka + '//GENERAL_PPI.shp', input_subsatellite_points=papka + '//subsatellite_points.shp'):    
    zrv_gdf = gpd.read_file(input_zrv)
    step = 0
    previous_state = 0
    ppi_session = []
    start_session = []
    end_session = []
    for id, row in zrv_gdf.iterrows():
        subsat_points_gdf = gpd.read_file(input_subsatellite_points)
        polygon_coordinates = row["geometry"]
        polygon = Polygon(polygon_coordinates)
        # start_gdf = gpd.GeoDataFrame()
        # end_gdf = gpd.GeoDataFrame()
        for id_point, subs_point in subsat_points_gdf.iterrows():
            x = float(subs_point.geometry.x)
            y = float(subs_point.geometry.y)
            point = Point(x, y)
            current_state = point.within(polygon)
            if current_state == 0 and previous_state == 0:
                subsat_points_gdf.drop(axis=0, index=id_point, inplace=True)
            elif current_state == 1 and previous_state == 0:
                start_session.append(subs_point["TIME"])
                ppi_session.append(id)
            elif current_state == 0 and previous_state == 1:
                end_session.append(subs_point["TIME"])
            previous_state = current_state
        if not subsat_points_gdf.empty:
            subsat_points_gdf.to_file(papka + '//intersected' + str(step) + '.shp')
            print('saved')
        step += 1
    session_data = dict(PPI=ppi_session, START=start_session, END=end_session)
    session_df = pd.DataFrame(session_data)
    session_df.to_excel(papka + '//SESSION_PPI.xlsx', index=False)

In [9]:
intersect_subsatellite_points_with_zrv()

saved
saved
